In [ ]:
# Install required packages
!pip install langchain-huggingface
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install langchain

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint
from google.colab import userdata
import os

In [ ]:
# Initialize HuggingFaceEndpoint with credentials
sec_key = userdata.get("HF_TOKEN")
os.environ["HF_TOKEN"] = sec_key
repo_id = "google/gemma-1.1-7b-it"
llm = HuggingFaceEndpoint(repo_id=repo_id, max_length=1024, temperature=0.7, token=sec_key)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


In [ ]:
# Function to generate job description and personalized advice
def generate_output(input_data):
    prompt = f"""
    [Role] You are a career advisor specializing in job recommendations for students who have completed courses at our institute.

    [Instruction] Please generate a suitable job description and personalized advice based on the following student's information:

    Enrollment Status: {input_data['EnrollmentStatus']}
    Attendance Percentage: {input_data['AttendancePercentage']}%
    Course Details:
        Course Name: {input_data['CourseDetails']['CourseName']}
        Modules Completed: {input_data['CourseDetails']['ModulesCompleted']} out of {input_data['CourseDetails']['TotalModules']}
    Quiz Status:
        Module 1: {input_data['QuizStatus']['Module1']}
        Module 2: {input_data['QuizStatus']['Module2']}
        Module 3: {input_data['QuizStatus']['Module3']}
        Module 4: {input_data['QuizStatus']['Module4']}
        Module 5: {input_data['QuizStatus']['Module5']}
        Module 6: {input_data['QuizStatus']['Module6']}
        Module 7: {input_data['QuizStatus']['Module7']}
    Assignment Completion: {input_data['AssignmentCompletion']['PercentageCompleted']}%
    Module Completion Certificate: {input_data['ModuleCompletionCertificate']}
    Course Exam Completion: {input_data['CourseExamCompletion']}
    LinkedIn Account: {'Yes' if input_data['LinkedInAccount'] else 'No'}
    LinkedIn Updated: {'Yes' if input_data['LinkedInUpdated'] else 'No'}
    Projects Completion: {input_data['ProjectsCompletion']}
    Viva Voce Completion:
        Attended: {input_data['VivaVoceCompletion']['Attended']}
        Passed: {input_data['VivaVoceCompletion']['Passed']}
    NASSCOM Exam: {'Yes' if input_data['NASSCOMExam'] else 'No'}
    SUNNY Exam: {'Yes' if input_data['SUNNYExam'] else 'No'}
    Research Paper Published: {'Yes' if input_data['ResearchPaperPublished'] else 'No'}
    Selected for Internship: {'Yes' if input_data['SelectedForInternship'] else 'No'}
    Resume Status: {'Yes' if input_data['ResumeStatus'] else 'No'}
    Internship Updated on LinkedIn: {'Yes' if input_data['InternshipUpdatedOnLinkedIn'] else 'No'}
    Job Platform Profiles:
        Indeed: {'Yes' if input_data['JobPlatformProfiles']['Indeed'] else 'No'}
        LinkedIn: {'Yes' if input_data['JobPlatformProfiles']['LinkedIn'] else 'No'}
        Glassdoor: {'Yes' if input_data['JobPlatformProfiles']['Glassdoor'] else 'No'}
        ZipRecruiter: {'Yes' if input_data['JobPlatformProfiles']['ZipRecruiter'] else 'No'}
        Monster: {'Yes' if input_data['JobPlatformProfiles']['Monster'] else 'No'}
        SimplyHired: {'Yes' if input_data['JobPlatformProfiles']['SimplyHired'] else 'No'}
    Resume Upload Status:
        Indeed: {'Yes' if input_data['ResumeUploadStatus']['Indeed'] else 'No'}
        LinkedIn: {'Yes' if input_data['ResumeUploadStatus']['LinkedIn'] else 'No'}
        Glassdoor: {'Yes' if input_data['ResumeUploadStatus']['Glassdoor'] else 'No'}
        ZipRecruiter: {'Yes' if input_data['ResumeUploadStatus']['ZipRecruiter'] else 'No'}
        Monster: {'Yes' if input_data['ResumeUploadStatus']['Monster'] else 'No'}
        SimplyHired: {'Yes' if input_data['ResumeUploadStatus']['SimplyHired'] else 'No'}

    [Output]

    Job Title: [Generated Job Title]
    Salary Range: [Optional, but helpful if data is available]

    Company: [Generated Company Name, Industry, Size, Culture]

    Responsibilities:
    1. [Generated Core duties and tasks]
    2. [Generated Additional responsibilities]
    3. [Generated Potential growth opportunities]

    Qualifications:
    1. [Generated Required education and experience]
    2. [Generated Essential skills and knowledge]
    3. [Generated Preferred qualifications (e.g., certifications, specific tools)]

    Additional Information: [Optional - Generated company culture fit, industry trends]

    Personalized Advice:
    {generate_personalized_advice(input_data)}
    """
    response = llm.invoke(prompt)
    return response

In [ ]:
# Function to generate personalized advice based on student data
def generate_personalized_advice(input_data):
    advice = ""

    # Attendance advice
    if input_data['AttendancePercentage'] < 80:
        advice += f"[Attendance is No]: It's important to attend classes regularly to fully grasp the course material. Consider reaching out to instructors for any missed content.\n"

    # Assignment completion advice
    if input_data['AssignmentCompletion']['PercentageCompleted'] < 80:
        advice += f"[If Assignments Completed is No]: Completing assignments is crucial for practicing skills. Review and finish pending assignments to solidify your knowledge.\n"

    # Module completion advice
    if input_data['CourseDetails']['ModulesCompleted'] < input_data['CourseDetails']['TotalModules']:
        advice += f"[If Modules Covered is No]: Make sure to cover all modules to gain a comprehensive understanding of the course. Refer to course materials or seek help if needed.\n"

    # Quiz completion advice
    if any(status != "Passed" for status in input_data['QuizStatus'].values()):
        advice += f"[If Quizzes Completed is No]: Quizzes assess your progress. Complete any pending quizzes to gauge your understanding and identify areas for improvement.\n"

    # Course completion certificate advice
    if not input_data['ModuleCompletionCertificate']:
        advice += f"[If Course Completion Certificate is No]: Ensure you meet all requirements to obtain the certificate, as it's often a prerequisite for job applications.\n"

    # Projects completion advice
    if input_data['ProjectsCompletion'] < 4:
        advice += f"[If Projects Completed is less than 4]: Aim to complete all projects to showcase your practical skills and enhance your portfolio.\n"

    # Viva voce advice


    # Resume advice
    if not input_data['ResumeStatus']:
        advice += f"[If Resume Created/Updated is No]: Create or update your resume to highlight your skills and experience effectively.\n"

    # Job platform accounts advice
    if not any(input_data['JobPlatformProfiles'].values()):
        advice += f"[If Job Platform Accounts is empty]: Create accounts on relevant job platforms to expand your reach and discover opportunities.\n"

    # Resume uploaded advice
    if not any(input_data['ResumeUploadStatus'].values()):
        advice += f"[If Resume Uploaded is No]: Upload your resume to all job platforms where you have accounts to increase your visibility to potential employers.\n"

    return advice.strip()

In [ ]:
# Create an example input dataset
student_data = {
    "EnrollmentStatus": "Enrolled",
    "AttendancePercentage": 85,
    "CourseDetails": {
        "CourseName": "Data Science",
        "ModulesCompleted": 7,
        "TotalModules": 10
    },
    "QuizStatus": {
        "Module1": "Passed",
        "Module2": "Passed",
        "Module3": "Passed",
        "Module4": "Passed",
        "Module5": "Not Attempted",
        "Module6": "Not Attempted",
        "Module7": "Passed"
    },
    "AssignmentCompletion": {
        "PercentageCompleted": 70
    },
    "ModuleCompletionCertificate": False,
    "CourseExamCompletion": False,
    "LinkedInAccount": True,
    "LinkedInUpdated": False,
    "ProjectsCompletion": 3,
    "VivaVoceCompletion": {
        "Attended": 2,
        "Passed": 1
    },
    "NASSCOMExam": False,
    "SUNNYExam": True,
    "ResearchPaperPublished": False,
    "SelectedForInternship": True,
    "ResumeStatus": True,
    "InternshipUpdatedOnLinkedIn": False,
    "JobPlatformProfiles": {
        "Indeed": True,
        "LinkedIn": True,
        "Glassdoor": False,
        "ZipRecruiter": False,
        "Monster": False,
        "SimplyHired": False
    },
    "ResumeUploadStatus": {
        "Indeed": True,
        "LinkedIn": True,
        "Glassdoor": False,
        "ZipRecruiter": False,
        "Monster": False,
        "SimplyHired": False
    }
}

In [ ]:
# Generate output
output = generate_output(student_data)
print(output)


```

**Job Title:** Data Analyst

**Salary Range:** ₹7-10 lakhs per annum

**Company:** Analytics Lab, Data Science & Analytics Consulting, Mumbai, India

**Responsibilities:**

- Collect, clean, and transform raw data from various sources.
- Analyze data using statistical methods and visualization tools.
- Identify patterns, trends, and insights from data.
- Develop and maintain data models and dashboards.
- Communicate findings and recommendations to stakeholders.

**Qualifications:**

- Bachelor's degree in Computer Science, Data Science, or related field.
- 1-2 years of experience in data analytics.
- Proficiency in SQL, Python, and Tableau.
- Excellent analytical and problem-solving skills.
- Strong communication and teamwork abilities.

**Additional Information:**

Analytics Lab fosters a data-driven culture and encourages continuous learning. The team works on projects across industries, including healthcare, finance, and e-commerce.

**Personalized Advice:**

- Ensure that you

In [ ]:
import re
import json

In [ ]:
# Job description prompt
prompt = """
**Job Title:** Data Analyst

**Salary Range:** ₹7-10 lakhs per annum

**Company:** Analytics Lab, Data Science & Analytics Consulting, Mumbai, India

**Responsibilities:**

- Collect, clean, and transform raw data from various sources.
- Analyze data using statistical methods and visualization tools.
- Identify patterns, trends, and insights from data.
- Develop and maintain data models and dashboards.
- Communicate findings and recommendations to stakeholders.

**Qualifications:**

- Bachelor's degree in Computer Science, Data Science, or related field.
- 1-2 years of experience in data analytics.
- Proficiency in SQL, Python, and Tableau.
- Excellent analytical and problem-solving skills.
- Strong communication and teamwork abilities.

**Additional Information:**

Analytics Lab fosters a data-driven culture and encourages continuous learning. The team works on projects across industries, including healthcare, finance, and e-commerce.

**Personalized Advice:**

- Ensure that you complete all assignments and quizzes to strengthen your foundational knowledge.
- Consider covering any remaining modules to gain a comprehensive understanding of the course.
- Enhance your portfolio by completing additional projects.
- Update your LinkedIn profile and internship experience.
- Network with professionals in the data analytics industry to learn about opportunities and gain insights.
"""

In [ ]:
# Regular expressions to match the required parts of the job description
regex_dict = {
    "Job Title": r"\*\*Job Title:\*\* (.+)",
    "Salary Range": r"\*\*Salary Range:\*\* (.+)",
    "Company": r"\*\*Company:\*\* (.+)",
    "Responsibilities": r"\*\*Responsibilities:\*\*([\s\S]+?)\*\*Qualifications:\*\*",
    "Qualifications": r"\*\*Qualifications:\*\*([\s\S]+?)\*\*Additional Information:\*\*",
    "Additional Information": r"\*\*Additional Information:\*\*([\s\S]+?)\*\*Personalized Advice:\*\*",
    "Personalized Advice": r"\*\*Personalized Advice:\*\*([\s\S]+)"
}

In [ ]:
# Function to extract data using regular expressions
def extract_data(prompt, regex_dict):
    extracted_data = {}
    for key, regex in regex_dict.items():
        match = re.search(regex, prompt)
        if match:
            extracted_data[key] = match.group(1).strip()
    return extracted_data

In [ ]:
# Extract data
extracted_data = extract_data(prompt, regex_dict)

In [ ]:
# Convert to JSON
extracted_data_json = json.dumps(extracted_data, indent=4)

In [ ]:
# Print the JSON result
print(extracted_data_json)

{
    "Job Title": "Data Analyst",
    "Salary Range": "\u20b97-10 lakhs per annum",
    "Company": "Analytics Lab, Data Science & Analytics Consulting, Mumbai, India",
    "Responsibilities": "- Collect, clean, and transform raw data from various sources.\n- Analyze data using statistical methods and visualization tools.\n- Identify patterns, trends, and insights from data.\n- Develop and maintain data models and dashboards.\n- Communicate findings and recommendations to stakeholders.",
    "Qualifications": "- Bachelor's degree in Computer Science, Data Science, or related field.\n- 1-2 years of experience in data analytics.\n- Proficiency in SQL, Python, and Tableau.\n- Excellent analytical and problem-solving skills.\n- Strong communication and teamwork abilities.",
    "Additional Information": "Analytics Lab fosters a data-driven culture and encourages continuous learning. The team works on projects across industries, including healthcare, finance, and e-commerce.",
    "Personal

In [1]:
pip install mysql-connector-python requests

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/14.5 MB 1.7 MB/s eta 0:00:09
   ---------------------------------------- 0.2/14.5 MB 2.6 MB/s eta 0:00:06
    --------------------------------------- 0.2/14.5 MB 1.8 MB/s eta 0:00:08
    --------------------------------------- 0.3/14.5 MB 2.0 MB/s eta 0:00:07
    --------------------------------------- 0.3/14.5 MB 2.0 MB/s eta 0:00:07
   - -------------------------------------- 0.5/14.5 MB 1.9 MB/s eta 0:00:08
   - -------------------------------------- 0.6/14.5 MB 2.2 MB/s eta 0:00:07
   - -------------------------------------- 0.7/14.5 MB 2.2 MB/s eta 0:00:07
   -- ------------------------------------- 1.1/14.5 MB 2.8 MB/s eta 0:00:05
   --- ------------------------------------ 1.2/14.5 MB 2.9 MB/s eta 0:00:05
   ---- ----------------------------------- 1.5/14.5 MB 3.3 MB/s eta 0:00:04
   --


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\akhil\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import mysql.connector
import json

# MySQL connection configuration
config = {
    'user': 'root',
    'password': 'akhila',
    'host': '127.0.0.1',
    'database': 'mydatabase',
    'raise_on_warnings': True
}

# Connect to the MySQL database
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

# Given JSON data
data = {
    "Job Title": "Data Analyst",
    "Salary Range": "\u20b97-10 lakhs per annum",
    "Company": "Analytics Lab, Data Science & Analytics Consulting, Mumbai, India",
    "Responsibilities": "- Collect, clean, and transform raw data from various sources.\n- Analyze data using statistical methods and visualization tools.\n- Identify patterns, trends, and insights from data.\n- Develop and maintain data models and dashboards.\n- Communicate findings and recommendations to stakeholders.",
    "Qualifications": "- Bachelor's degree in Computer Science, Data Science, or related field.\n- 1-2 years of experience in data analytics.\n- Proficiency in SQL, Python, and Tableau.\n- Excellent analytical and problem-solving skills.\n- Strong communication and teamwork abilities.",
    "Additional Information": "Analytics Lab fosters a data-driven culture and encourages continuous learning. The team works on projects across industries, including healthcare, finance, and e-commerce.",
    "Personalized Advice": "- Ensure that you complete all assignments and quizzes to strengthen your foundational knowledge.\n- Consider covering any remaining modules to gain a comprehensive understanding of the course.\n- Enhance your portfolio by completing additional projects.\n- Update your LinkedIn profile and internship experience.\n- Network with professionals in the data analytics industry to learn about opportunities and gain insights."
}

# SQL query to insert data
query = """
INSERT INTO job_info (job_title, salary_range, company, responsibilities, qualifications, additional_information, personalized_advice)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

# Values to be inserted
values = (
    data["Job Title"],
    data["Salary Range"],
    data["Company"],
    data["Responsibilities"],
    data["Qualifications"],
    data["Additional Information"],
    data["Personalized Advice"]
)

# Execute the query and commit the transaction
cursor.execute(query, values)
conn.commit()

# Close the connection
cursor.close()
conn.close()